In [1]:
import pandas as pd

<ipython-input-1-7dd3504c366f>:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
matches = pd.read_csv("matches.csv", index_col=0)

In [3]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


In [4]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk              float64
pkatt           float64
season            int64
team             object
dtype: object

In [5]:
matches["date"] = pd.to_datetime(matches["date"]) #converting date column (object) to datetime (float64) 

In [6]:
matches.dtypes

date            datetime64[ns]
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
gf                     float64
ga                     float64
opponent                object
xg                     float64
xga                    float64
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                  float64
sh                     float64
sot                    float64
dist                   float64
fk                     float64
pk                     float64
pkatt                  float64
season                   int64
team                    object
dtype: object

In [7]:
#here we are creating the predictor columns that are going to be used to predict the outcome of the games

In [8]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes #converts venue_code (object) to categorical type || .cat.codes converts the values to integers

In [9]:
matches["opp_code"] = matches["opponent"].astype("category").cat.codes #creates column called opp_code to give opponent team a unique code

In [10]:
matches["hour"] = matches["time"].str.replace(":.+","",regex=True).astype("int") #what if teams play better at certain hours: need to keep track of this by droping the minutes and ':' in the time column

In [11]:
matches["day_code"] = matches["date"].dt.dayofweek

In [12]:
#setting up machine model

In [13]:
matches["target"] = (matches["result"] == "W")

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
rf = RandomForestClassifier(n_estimators=50,min_samples_split=10, random_state=1)

In [16]:
train = matches[matches["date"] < '2022-01-01']

In [17]:
test = matches[matches["date"] > '2022-01-01']

In [18]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [19]:
rf.fit(train[predictors],train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [20]:
preds = rf.predict(test[predictors])

In [21]:
#determine accuracy of model
from sklearn.metrics import accuracy_score #if you predict 'W' what % of time did the team actually win; if predict 'L' what % of time did the team actually lose

In [22]:
acc = accuracy_score(test["target"],preds)

In [23]:
acc

0.6123188405797102

In [24]:
#now we can look into which situations was our accuracy high verus low

In [25]:
combined = pd.DataFrame(dict(actual=test["target"],prediction=preds))

In [26]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])  #crosstab is being used to show us when we predicted a false or true  to what actually happened

prediction,False,True
actual,,
False,141,31
True,76,28


In [27]:
#model is predicting more incorrectly than correctly when the prediction s 'true', therefore revising accuracy metric is necessary

In [28]:
from sklearn.metrics import precision_score #this is gonna tell us that when the model predicts a win, what % of the time was the actual result a win

In [29]:
precision_score(test["target"],preds) #47% of the time the predcition was correct (BEFORE)

0.4745762711864407

In [30]:
#more predictors

In [31]:
grouped_matches = matches.groupby("team") #group matches by teams

In [32]:
group = grouped_matches.get_group("Manchester City") #example of how matches are grouped

In [33]:
group

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,1.0,0.0,0.0,2022,Manchester City,0,18,16,6,False
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,1.0,0.0,0.0,2022,Manchester City,1,15,15,5,True
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,0.0,0.0,0.0,2022,Manchester City,1,0,12,5,True
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,0.0,0.0,0.0,2022,Manchester City,0,10,15,5,True
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,1.0,0.0,0.0,2022,Manchester City,1,17,15,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,2021-05-01,12:30,Premier League,Matchweek 34,Sat,Away,W,2.0,0.0,Crystal Palace,...,1.0,0.0,0.0,2021,Manchester City,0,6,12,5,True
56,2021-05-08,17:30,Premier League,Matchweek 35,Sat,Home,L,1.0,2.0,Chelsea,...,0.0,0.0,1.0,2021,Manchester City,1,5,17,5,False
57,2021-05-14,20:00,Premier League,Matchweek 36,Fri,Away,W,4.0,3.0,Newcastle Utd,...,1.0,0.0,0.0,2021,Manchester City,0,14,20,4,True
58,2021-05-18,19:00,Premier League,Matchweek 37,Tue,Away,L,2.0,3.0,Brighton,...,1.0,0.0,0.0,2021,Manchester City,0,3,19,1,False


In [34]:
def rolling_averages(group, cols, new_cols): #takes in group, a set of cols we want to compute rolling avgs for, put avgs into new_cols
    
    group = group.sort_values("date") #sort by asc order to see last 3 matches played and see their performance
    
    rolling_stats = group[cols].rolling(3,closed='left').mean() #computes rolling avgs for the cols, will put the data into the third col, closed='left' -> important: w/o this pandas will take in future data
    #ex prediciting week 3 matches, w/o closed='left' pandas will include 1,2, AND the future data of week 3 (data in week 3 contains nothing for this example) 
    #*****TAKE THE CURRENT WEEK OUT******

    group[new_cols] = rolling_stats

    group = group.dropna(subset=new_cols) # drop missing values, w/o this pandas will put "MISSING VALUES" in data which we do not want in our data
    return group
    

In [35]:
cols = ["gf","ga","sh","sot","dist","fk","pk","pkatt"] #stats that are for previous three matches
new_cols= [f"{c}_rolling" for c in cols] #format string that adds 'rolling' to the end of column names

In [36]:
new_cols

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [37]:
rolling_averages(group, cols, new_cols)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
5,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Home,W,1.0,0.0,Arsenal,...,5,True,2.000000,2.333333,17.333333,4.666667,18.900000,1.333333,0.333333,0.333333
7,2020-10-24,12:30,Premier League,Matchweek 6,Sat,Away,D,1.0,1.0,West Ham,...,5,False,1.333333,2.000000,17.333333,3.666667,17.733333,0.666667,0.000000,0.000000
9,2020-10-31,12:30,Premier League,Matchweek 7,Sat,Away,W,1.0,0.0,Sheffield Utd,...,5,True,1.000000,0.666667,16.666667,4.333333,18.233333,0.666667,0.000000,0.000000
11,2020-11-08,16:30,Premier League,Matchweek 8,Sun,Home,D,1.0,1.0,Liverpool,...,6,False,1.000000,0.333333,14.333333,6.666667,18.466667,1.000000,0.000000,0.000000
12,2020-11-21,17:30,Premier League,Matchweek 9,Sat,Away,L,0.0,2.0,Tottenham,...,5,False,1.000000,0.666667,12.000000,5.666667,19.366667,1.000000,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2022-03-14,20:00,Premier League,Matchweek 29,Mon,Away,D,0.0,0.0,Crystal Palace,...,0,False,2.333333,1.333333,19.000000,7.000000,15.366667,0.333333,0.333333,0.333333
44,2022-04-02,15:00,Premier League,Matchweek 31,Sat,Away,W,2.0,0.0,Burnley,...,5,True,1.666667,0.333333,18.333333,7.333333,16.000000,0.333333,0.000000,0.000000
46,2022-04-10,16:30,Premier League,Matchweek 32,Sun,Home,D,2.0,2.0,Liverpool,...,6,False,2.000000,0.333333,20.000000,6.666667,16.133333,0.333333,0.000000,0.000000
49,2022-04-20,20:00,Premier League,Matchweek 30,Wed,Home,W,3.0,0.0,Brighton,...,2,True,1.333333,0.666667,15.666667,4.666667,16.700000,0.333333,0.000000,0.000000


In [38]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x,cols,new_cols))
# we group the data up by the "team" column and then use .apply to apply the rolling_averages function to each team

<ipython-input-38-dda2dbd93114>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x,cols,new_cols))


In [39]:
matches_rolling

date   time            comp         round  \
team                                                                         
Arsenal                 6  2020-10-04  14:00  Premier League   Matchweek 4   
                        7  2020-10-17  17:30  Premier League   Matchweek 5   
                        9  2020-10-25  19:15  Premier League   Matchweek 6   
                        11 2020-11-01  16:30  Premier League   Matchweek 7   
                        13 2020-11-08  19:15  Premier League   Matchweek 8   
...                               ...    ...             ...           ...   
Wolverhampton Wanderers 32 2022-03-13  14:00  Premier League  Matchweek 29   
                        33 2022-03-18  20:00  Premier League  Matchweek 30   
                        34 2022-04-02  15:00  Premier League  Matchweek 31   
                        35 2022-04-08  20:00  Premier League  Matchweek 32   
                        36 2022-04-24  14:00  Premier League  Matchweek 34   

                            day venue result   gf   ga         opponent  ...  \
team                                                                     ...   
Arsenal                 6   Sun  Home      W  2.0  1.0    Sheffield Utd  ...   
                        7   Sat  Away      L  0.0  1.0  Manchester City  ...   
                        9   Sun  Home      L  0.0  1.0   Leicester City  ...   
                        11  Sun  Away      W  1.0  0.0   Manchester Utd  ...   
                        13  Sun  Home      L  0.0  3.0      Aston Villa  ...   
...                         ...   ...    ...  ...  ...              ...  ...   
Wolverhampton Wanderers 32  Sun  Away      W  1.0  0.0          Everton  ...   
                        33  Fri  Home      L  2.0  3.0     Leeds United  ...   
                        34  Sat  Home      W  2.0  1.0      Aston Villa  ...   
                        35  Fri  Away      L  0.0  1.0    Newcastle Utd  ...   
                        36  Sun  Away      L  0.0  1.0          Burnley  ...   

                            day_code  target  gf_rolling  ga_rolling  \
team                                                                   
Arsenal                 6          6    True    2.000000    1.333333   
                        7          5   False    1.666667    1.666667   
                        9          6   False    1.000000    1.666667   
                        11         6    True    0.666667    1.000000   
                        13         6   False    0.333333    0.666667   
...                              ...     ...         ...         ...   
Wolverhampton Wanderers 32         6    True    1.333333    1.000000   
                        33         4   False    1.666667    0.666667   
                        34         5    True    2.333333    1.000000   
                        35         4   False    1.666667    1.333333   
                        36         6   False    1.333333    1.666667   

                           sh_rolling sot_rolling dist_rolling fk_rolling  \
team                                                                        
Arsenal                 6    7.666667    3.666667    14.733333   0.666667   
                        7    5.333333    3.666667    15.766667   0.000000   
                        9    7.000000    3.666667    16.733333   0.666667   
                        11   9.666667    4.000000    16.033333   1.000000   
                        13   9.666667    2.666667    18.033333   1.000000   
...                               ...         ...          ...        ...   
Wolverhampton Wanderers 32  12.333333    3.666667    19.300000   0.000000   
                        33  12.333333    4.333333    19.600000   0.000000   
                        34  13.000000    5.333333    19.833333   0.000000   
                        35  13.000000    5.000000    18.533333   0.000000   
                        36  10.000000    4.666667    17.633333   0.000000   

                            pk_rolling

In [40]:
matches_rolling = matches_rolling.droplevel("team") #drops team column since it is not a necessary column to have

In [41]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
6,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,Sheffield Utd,...,6,True,2.000000,1.333333,7.666667,3.666667,14.733333,0.666667,0.000000,0.000000
7,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,Manchester City,...,5,False,1.666667,1.666667,5.333333,3.666667,15.766667,0.000000,0.000000,0.000000
9,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0.0,1.0,Leicester City,...,6,False,1.000000,1.666667,7.000000,3.666667,16.733333,0.666667,0.000000,0.000000
11,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1.0,0.0,Manchester Utd,...,6,True,0.666667,1.000000,9.666667,4.000000,16.033333,1.000000,0.000000,0.000000
13,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0.0,3.0,Aston Villa,...,6,False,0.333333,0.666667,9.666667,2.666667,18.033333,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,2022-03-13,14:00,Premier League,Matchweek 29,Sun,Away,W,1.0,0.0,Everton,...,6,True,1.333333,1.000000,12.333333,3.666667,19.300000,0.000000,0.000000,0.000000
33,2022-03-18,20:00,Premier League,Matchweek 30,Fri,Home,L,2.0,3.0,Leeds United,...,4,False,1.666667,0.666667,12.333333,4.333333,19.600000,0.000000,0.000000,0.000000
34,2022-04-02,15:00,Premier League,Matchweek 31,Sat,Home,W,2.0,1.0,Aston Villa,...,5,True,2.333333,1.000000,13.000000,5.333333,19.833333,0.000000,0.000000,0.000000
35,2022-04-08,20:00,Premier League,Matchweek 32,Fri,Away,L,0.0,1.0,Newcastle Utd,...,4,False,1.666667,1.333333,13.000000,5.000000,18.533333,0.000000,0.000000,0.000000


In [42]:
#1317 rows but only 36 are showing -> lots of data is being repeating, we only want unique values 

In [43]:
matches_rolling.index = range(matches_rolling.shape[0]) #this will assign values to indicies from 0-1316

In [44]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,Sheffield Utd,...,6,True,2.000000,1.333333,7.666667,3.666667,14.733333,0.666667,0.000000,0.000000
1,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,Manchester City,...,5,False,1.666667,1.666667,5.333333,3.666667,15.766667,0.000000,0.000000,0.000000
2,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0.0,1.0,Leicester City,...,6,False,1.000000,1.666667,7.000000,3.666667,16.733333,0.666667,0.000000,0.000000
3,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1.0,0.0,Manchester Utd,...,6,True,0.666667,1.000000,9.666667,4.000000,16.033333,1.000000,0.000000,0.000000
4,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0.0,3.0,Aston Villa,...,6,False,0.333333,0.666667,9.666667,2.666667,18.033333,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,2022-03-13,14:00,Premier League,Matchweek 29,Sun,Away,W,1.0,0.0,Everton,...,6,True,1.333333,1.000000,12.333333,3.666667,19.300000,0.000000,0.000000,0.000000
1313,2022-03-18,20:00,Premier League,Matchweek 30,Fri,Home,L,2.0,3.0,Leeds United,...,4,False,1.666667,0.666667,12.333333,4.333333,19.600000,0.000000,0.000000,0.000000
1314,2022-04-02,15:00,Premier League,Matchweek 31,Sat,Home,W,2.0,1.0,Aston Villa,...,5,True,2.333333,1.000000,13.000000,5.333333,19.833333,0.000000,0.000000,0.000000
1315,2022-04-08,20:00,Premier League,Matchweek 32,Fri,Away,L,0.0,1.0,Newcastle Utd,...,4,False,1.666667,1.333333,13.000000,5.000000,18.533333,0.000000,0.000000,0.000000


In [45]:
def make_predications(data,predictors): #function that makes it easier for us to build upon the algortihm we are using and lets us add the new predictors to the algorithm
    #split data into training set and test set
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    precision = precision_score(test["target"],preds)
    return combined, precision

In [46]:
combined, precision = make_predications(matches_rolling,predictors + new_cols)

In [47]:
precision

0.625

In [48]:
#precision has increased due to new predictor columns, could it be further precise with more columns?

In [49]:
combined

,actual,predicted
55,False,False
56,True,False
57,True,False
58,True,True
59,True,True
...,...,...
1312,True,False
1313,False,False
1314,True,False
1315,False,False


In [50]:
#though combined contains the results of the predictions and if they were accurate, it doesnt tell the team of the prediciton
#to fix this, we can use .merge to merge the date, team, oppoenet, result columns

In [51]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True) # matches the column index of combined with the indices in matches_rolling

In [52]:
combined

,actual,predicted,date,team,opponent,result
55,False,False,2022-01-23,Arsenal,Burnley,D
56,True,False,2022-02-10,Arsenal,Wolves,W
57,True,False,2022-02-19,Arsenal,Brentford,W
58,True,True,2022-02-24,Arsenal,Wolves,W
59,True,True,2022-03-06,Arsenal,Watford,W
...,...,...,...,...,...,...
1312,True,False,2022-03-13,Wolverhampton Wanderers,Everton,W
1313,False,False,2022-03-18,Wolverhampton Wanderers,Leeds United,L
1314,True,False,2022-04-02,Wolverhampton Wanderers,Aston Villa,W
1315,False,False,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L


In [53]:
#since the algorithm predicted both sides of the match, we are going to combine those rows since it might have made diff predictions. This can help us dig deeper into the algorithm and well as fix any underlying issues

In [54]:
#first we are going to normalize the names of the teams due to some columns having a different name for a team than others
class MissingDict(dict): # creating a class that inherits from dict class, this is so that pandas can handle missing keys, what pandas will do is remove the name from dict instead of replacing with original name
    __missing__ = lambda self, key: key

map_values = {
    "Brighton and Hove Albion": "righton",
    "Manchester United": "Machester Utd",
    "Newcastle United:": "Newcastle Utd",
    "Tottenham Hotspur": "tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"
}# we are only putting the names that are inconsistent in the dict
mapping = MissingDict(**map_values)

In [55]:
mapping["West Ham United"]

'West Ham'

In [56]:
combined["new_team"] = combined["team"].map(mapping)

In [57]:
combined

,actual,predicted,date,team,opponent,result,new_team
55,False,False,2022-01-23,Arsenal,Burnley,D,Arsenal
56,True,False,2022-02-10,Arsenal,Wolves,W,Arsenal
57,True,False,2022-02-19,Arsenal,Brentford,W,Arsenal
58,True,True,2022-02-24,Arsenal,Wolves,W,Arsenal
59,True,True,2022-03-06,Arsenal,Watford,W,Arsenal
...,...,...,...,...,...,...,...
1312,True,False,2022-03-13,Wolverhampton Wanderers,Everton,W,Wolves
1313,False,False,2022-03-18,Wolverhampton Wanderers,Leeds United,L,Wolves
1314,True,False,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,Wolves
1315,False,False,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,Wolves


In [58]:
#now we can use new_team column to merge the dataframe

In [59]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [60]:
merged[(merged["predicted_x"] == False) & (merged["predicted_y"] == True)]["actual_x"].value_counts()

actual_x
False    26
True      6
Name: count, dtype: int64

In [61]:
#merged = combined.merge(combined, left_on=["date","new_team"], right_on=["date","opponent"]) #merging the dataframe with itself, or just matching the columns together

In [62]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,False,False,2022-01-23,Arsenal,Burnley,D,Arsenal,False,False,Burnley,Arsenal,D,Burnley
1,True,False,2022-02-10,Arsenal,Wolves,W,Arsenal,False,False,Wolverhampton Wanderers,Arsenal,L,Wolves
2,True,False,2022-02-19,Arsenal,Brentford,W,Arsenal,False,False,Brentford,Arsenal,L,Brentford
3,True,True,2022-02-24,Arsenal,Wolves,W,Arsenal,False,False,Wolverhampton Wanderers,Arsenal,L,Wolves
4,True,True,2022-03-06,Arsenal,Watford,W,Arsenal,False,False,Watford,Arsenal,L,Watford
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,True,False,2022-03-13,Wolverhampton Wanderers,Everton,W,Wolves,False,False,Everton,Wolves,L,Everton
198,False,False,2022-03-18,Wolverhampton Wanderers,Leeds United,L,Wolves,True,False,Leeds United,Wolves,W,Leeds United
199,True,False,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,Wolves,False,False,Aston Villa,Wolves,L,Aston Villa
200,False,False,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,True,False,Newcastle United,Wolves,W,Newcastle United


In [63]:
26/32

0.8125